### DESCRIPTION:
    This example shows how to retrieve data from Azure SQL DB by using Open AI GPT.  
    Asking questions in plain english that gets "translated" by GPT into SQL.
    Using Langchain SQLDatabaseChain 
### REQUIREMENTS:
    1. Create an Azure SQL DB and populate it with data.
    2. Create an .env file in the root folder with the following variables:
      SQL_SERVER="<server>"
      SQL_USER="<user>"
      SQL_PWD="<pwd>"
      SQL_DBNAME="<dbname>"

### Sample questions you can ask:
      List the tables in the database
      How many products are in the Adventure Works database?
      How many Products are color black?
      How many SalesOrderDetail are for the Product AWC Logo Cap ?
      List the top 10 most expensive products
      What are the top 10 highest grossing products in the Adventure Works database?

### For more information about Langchain agent toolkits, see:
  https://github.com/hwchase17/langchain/tree/master/langchain/agents/agent_toolkits


In [1]:
from langchain.llms import AzureOpenAI
from langchain.chat_models import ChatOpenAI
from langchain.utilities import SQLDatabase
from langchain.chains import create_sql_query_chain
from dotenv import load_dotenv
import openai
import os

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") 
OPENAI_DEPLOYMENT_ENDPOINT = os.getenv("OPENAI_DEPLOYMENT_ENDPOINT")
OPENAI_DEPLOYMENT_NAME = os.getenv("OPENAI_DEPLOYMENT_NAME")
OPENAI_MODEL_NAME = os.getenv("OPENAI_MODEL_NAME")
OPENAI_DEPLOYMENT_VERSION = os.getenv("OPENAI_DEPLOYMENT_VERSION")

SQL_SERVER = os.getenv("SQL_SERVER")
SQL_USER = os.getenv("SQL_USER")
SQL_PWD = os.getenv("SQL_PWD")
SQL_DBNAME = os.getenv("SQL_DBNAME")

# Configure OpenAI API
openai.api_type = "azure"
openai.api_version = OPENAI_DEPLOYMENT_VERSION
openai.api_base = OPENAI_DEPLOYMENT_ENDPOINT
openai.api_key = OPENAI_API_KEY

In [8]:
def init_llm(model=OPENAI_MODEL_NAME,
             deployment_name=OPENAI_DEPLOYMENT_NAME,
             openai_api_version=OPENAI_DEPLOYMENT_VERSION,
             temperature=0,
             max_tokens=400,
             top_p=1,
             ):

    llm = AzureOpenAI(deployment_name=deployment_name,
                      model=model,
                      openai_api_version=openai_api_version,
                      base_url=OPENAI_DEPLOYMENT_ENDPOINT,
                      temperature=temperature,
                      max_tokens=max_tokens,
                      top_p=top_p,
                    #   model_kwargs={"stop": ["<|im_end|>"]}
                      )
    return llm
llm = init_llm()

c:\Users\dschlesinger\code\ongoing\workshops\AOAI-workshop\.venv\lib\site-packages\langchain\llms\openai.py:853: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://aidemos-workshop.openai.azure.com/ to https://aidemos-workshop.openai.azure.com/openai.
  warnings.warn(
c:\Users\dschlesinger\code\ongoing\workshops\AOAI-workshop\.venv\lib\site-packages\langchain\llms\openai.py:860: UserWarning: As of openai>=1.0.0, if `deployment_name` (or alias `azure_deployment`) is specified then `openai_api_base` (or alias `base_url`) should not be. Instead use `deployment_name` (or alias `azure_deployment`) and `azure_endpoint`.
  warnings.warn(
c:\Users\dschlesinger\code\ongoing\workshops\AOAI-workshop\.venv\lib\site-packages\langchain\llms\openai.py:868: UserWarning: As of openai>=1.0.0, if `openai_api_base` (or alias `base_url`) is specified it is expected to be of

### Generate SQL and then run in DB

In [9]:
# initialize azure OpenAI
sqlconn = f"mssql+pymssql://{SQL_USER}:{SQL_PWD}@{SQL_SERVER}:1433/{SQL_DBNAME}"
db = SQLDatabase.from_uri(sqlconn)
chain = create_sql_query_chain(llm, db)

In [12]:
# generate SQL from question in English
query = chain.invoke({"question":"How many products are in the Adventure Works database using SalesLT schema?"})
print("Query generated by OpenAI: " + query)

Query generated by OpenAI: SELECT COUNT(*) FROM [SalesLT].[Product]


In [13]:
#run generated SQL in DB
db.run(query)

'[(295,)]'

In [14]:
# generate SQL from question in English
query = chain.invoke({"question":"List the top 10 most expensive products using SalesLT schema"})
print("Query generated by OpenAI: " + query)

Query generated by OpenAI: SELECT TOP 10 [Name], [ListPrice] FROM [SalesLT].[Product] ORDER BY [ListPrice] DESC


In [15]:
#run generated SQL in DB
db.run(query)

"[('Road-150 Red, 62', Decimal('3578.2700')), ('Road-150 Red, 44', Decimal('3578.2700')), ('Road-150 Red, 48', Decimal('3578.2700')), ('Road-150 Red, 52', Decimal('3578.2700')), ('Road-150 Red, 56', Decimal('3578.2700')), ('Mountain-100 Silver, 38', Decimal('3399.9900')), ('Mountain-100 Silver, 42', Decimal('3399.9900')), ('Mountain-100 Silver, 44', Decimal('3399.9900')), ('Mountain-100 Silver, 48', Decimal('3399.9900')), ('Mountain-100 Black, 38', Decimal('3374.9900'))]"